In [1]:
import pandas as pd
import numpy as np
import pickle
import pandas as pd
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import itertools
from collections import Counter
import numpy as np
from nltk import word_tokenize
from nltk.corpus import stopwords
from gensim.models import word2vec
from sklearn.linear_model import LogisticRegression
import os
import string

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
import os
import numpy as np
from gensim.models import word2vec
import re
import en_core_web_sm


In [3]:
data_path = "/home/jovyan/Documents/MSBA/unstrucured_data/mgta-415-winter2024/"

df_train = pd.read_csv(data_path + "train.csv")
df_test = pd.read_csv(data_path + "test.csv")

# Data preprocessing

In [4]:
import re
import string
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import spacy
import en_core_web_sm


nlp = en_core_web_sm.load()

def preprocess(doc, stemming=True, need_sent=False, use_spacy=False):
    # Step 0: Prepare resources
    stop_words = set(stopwords.words('english'))
    stop_words.add('would')
    ps = PorterStemmer()
    translator = str.maketrans(string.punctuation, ' ' * len(string.punctuation))
    
    # Step 1: Lowercase and remove punctuation
    doc = doc.lower().translate(translator)
    
    # Step 2 & 3: Tokenize into sentences, then words, and apply stemming if needed
    sentences = sent_tokenize(doc)
    processed_tokens = []
    for sent in sentences:
        words = word_tokenize(sent)
        if stemming:
            words = [ps.stem(word) for word in words]
        words = [word for word in words if word not in stop_words and len(word) > 1]
        if need_sent:
            processed_tokens.append(words)
        else:
            processed_tokens.extend(words)
    
    # Optional: Use spaCy for further processing (like lemmatization or entity recognition)
    if use_spacy:
        # Since spaCy expects a string, join tokens for spaCy processing and then re-tokenize if needed
        spacy_doc = nlp(" ".join(processed_tokens))
        # Example: Replace processed_tokens with lemma. Modify as needed for your use case
        processed_tokens = [token.lemma_ for token in spacy_doc]
        if need_sent:
            # If sentence structure was required, this needs rethinking based on spaCy's output
            raise NotImplementedError("Combining spaCy processing with sentence-wise token output not directly supported.")
    
    return processed_tokens

"""
# Example usage:
df['processed_text'] = df['text'].apply(lambda x: ' '.\
                                        join(combined_preprocess(x, stemming=True, need_sent=False, use_spacy=True)))
"""

"\n# Example usage:\ndf['processed_text'] = df['text'].apply(lambda x: ' '.                                        join(combined_preprocess(x, stemming=True, need_sent=False, use_spacy=True)))\n"

In [10]:
df_train['text'] =df_train['review'].apply(lambda x: ' '.\
                                        join(preprocess(x, stemming=True, need_sent=False, use_spacy=False)))

df_test['text'] =df_test['review'].apply(lambda x: ' '.\
                                        join(preprocess(x, stemming=True, need_sent=False, use_spacy=False)))

In [16]:
import pickle

# Save the preprocessed DataFrame to a Pickle file
with open('df_train_preprocessed.pkl', 'wb') as file:
    pickle.dump(df_train, file)

with open('df_test_preprocessed.pkl', 'wb') as file:
    pickle.dump(df_test, file)


In [11]:
# tokenization
tagged_data = [word_tokenize(_d) for i, _d in enumerate(df_train["text"])]
# build vocabulary from tokenized data
word_counts, vocabulary, vocabulary_inv = build_vocab(tagged_data)
# use the above mapping to create input data
inp_data = [[vocabulary[word] for word in text] for text in tagged_data]

# get embedding vector
#embedding_weights = get_embeddings(inp_data, vocabulary_inv)

In [1]:
from tensorflow import keras

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

vocabulary = 10000
tokenizer = Tokenizer(num_words=vocabulary)

# Tokenize the preprocessed texts
texts_train = df_train['text'].tolist()
texts_test = df_test['text'].tolist()

tokenizer.fit_on_texts(texts_train)

# Convert texts to sequences
sequence_train = tokenizer.texts_to_sequences(texts_train)
sequence_test = tokenizer.texts_to_sequences(texts_test)

# Define the maximum number of words in a sequence
word_num = 350

# Pad sequences to ensure uniform length
x_train = pad_sequences(sequence_train, maxlen=word_num)
x_test = pad_sequences(sequence_test, maxlen=word_num)

In [8]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

labels_train = df_train['label'].values
label_encoder = LabelEncoder()
labels_train_int = label_encoder.fit_transform(labels_train)
labels_train = to_categorical(labels_train_int, num_classes=10)

labels_train

array([[0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [9]:
# Split the training data into training and validation sets
x_train, x_valid, labels_train, labels_valid = train_test_split(
    x_train, labels_train, test_size=0.20, random_state=42)

In [ ]:
"""
for feature
array([[   0,    0,    0, ...,    3,   61,   74],
       [   0,    0,    0, ...,  324,  170,  238],
       [   0,    0,    0, ...,  561, 4440,  229],
       ...,
       [   0,    0,    0, ...,   16,    7,  438],
       [   0,    0,    0, ...,  340,  202,  144],
       [   0,    0,    0, ...,  588,  448, 4507]], dtype=int32)

for labels
array([[0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0]])
"""

# Tune the hyperparameter

In [3]:
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from keras_tuner import HyperModel, BayesianOptimization

class LSTMHyperModel(HyperModel):
    def __init__(self, vocabulary, word_num, output_units):
        self.vocabulary = vocabulary
        self.word_num = word_num
        self.output_units = output_units

    def build(self, hp):
        
        embedding_dim = hp.Int('embedding_dim', min_value=16, max_value=128, step=16)
        
        model = Sequential()
        model.add(Embedding(self.vocabulary, embedding_dim, input_length=self.word_num))
        model.add(LSTM(units=hp.Int('units_1', min_value=32, max_value=512, step=32), return_sequences=True))
        model.add(LSTM(units=hp.Int('units_2', min_value=16, max_value=512, step=32), return_sequences=False))
        model.add(Dense(self.output_units, activation='softmax'))

        hp_learning_rate = hp.Choice('learning_rate', values=[1e-3])

        model.compile(optimizer=RMSprop(learning_rate=hp_learning_rate),
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])
        return model

hypermodel = LSTMHyperModel(vocabulary=10000, word_num=350, output_units=10)

tuner = BayesianOptimization(
    hypermodel,
    objective='val_accuracy',
    max_trials=10,
    num_initial_points=2,
    directory='my_dir',
    project_name='lstm_bayesian_tuning_embedding_dim'
)


Reloading Tuner from my_dir/lstm_bayesian_tuning_embedding_dim/tuner0.json


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    monitor='val_accuracy',  
    patience=5,              
    restore_best_weights=True
)

tuner.search(
    x_train, labels_train,
    epochs=10,
    validation_data=(x_valid, labels_valid),
    callbacks=[early_stopping]  
)

In [21]:
best_model = tuner.get_best_models(num_models=1)[0]
best_hyperparameters = tuner.get_best_hyperparameters()[0]

print('Best model summary:')
best_model.summary()
print('Best hyperparameters:', best_hyperparameters.values)

Best model summary:
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 350, 32)           320000    
                                                                 
 lstm (LSTM)                 (None, 350, 320)          451840    
                                                                 
 lstm_1 (LSTM)               (None, 240)               538560    
                                                                 
 dense (Dense)               (None, 10)                2410      
                                                                 
Total params: 1312810 (5.01 MB)
Trainable params: 1312810 (5.01 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Best hyperparameters: {'embedding_dim': 32, 'units_1': 320, 'units_2': 240, 'learning_rate': 0.001}


# Train the network using 5 fold CV

In [ ]:
"""
from sklearn.model_selection import KFold
import numpy as np

# Define the K-Fold Cross-Validator
num_folds = 5
kfold = KFold(n_splits=num_folds, shuffle=True, random_state=42)

from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

fold_no = 1
acc_per_fold = []
loss_per_fold = []

for train, test in kfold.split(x_train, labels_train):
    # Clone the best model
    model = best_model  # Assuming `best_model` is already defined

    # Generate a print
    print(f'Training for fold {fold_no} ...')

    # Fit data to model
    history = model.fit(x_train[train], labels_train[train],
                        batch_size=32,
                        epochs=10,
                        verbose=1,
                        validation_split=0.2,  # Consider adjusting this value
                        callbacks=[early_stopping])

    # Generate generalization metrics
    scores = model.evaluate(x_train[test], labels_train[test], verbose=0)
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

    # Increase fold number
    fold_no = fold_no + 1

#== Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(len(acc_per_fold)):
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
avg_loss = np.mean(loss_per_fold)
avg_acc = np.mean(acc_per_fold)
print(f'Average loss across all folds: {avg_loss}')
print(f'Average accuracy across all folds: {avg_acc}%')
"""

In [ ]:
# Assuming you have separate validation data: x_val, y_val
best_model.fit(x_train, labels_train, epochs=10, batch_size=32, 
               validation_data=(x_valid, labels_valid), 
               callbacks=[early_stopping], 
               verbose=1)

In [23]:
# Now, predict x_test with the retrained model
predictions = best_model.predict(x_test)

# For multi-class classification, convert probabilities to class labels
predicted_classes = np.argmax(predictions, axis=1)

# If necessary, convert integer labels back to original class labels using LabelEncoder
predicted_labels = label_encoder.inverse_transform(predicted_classes)

313/313 [==============================] - 82s 262ms/step


In [24]:
dic = {"Id": [], "Predicted": []}
for i, pred in enumerate(predicted_labels):
    dic["Id"].append(i)
    dic["Predicted"].append(pred)

dic_df = pd.DataFrame.from_dict(dic)
dic_df.to_csv(data_path + "predicted_nn.csv", index=False)